In [ ]:
from dlclive import DLCLive, Processor
import dlclive
from dlclive.display import Display
import cv2
import numpy as np 

In [ ]:
# #somewhat works, but doesn't sve keypoints on video
# def analyze_video2(video_path: str, dlc_live):
#     # Load video
#     cap = cv2.VideoCapture(video_path)
#     poses = []
#     frame_index = 0

#     ###

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break  # End of video

#         # Prepare the frame for the model
#         print("a")
#         frame = np.array(frame, dtype=np.float32)
#         print("b")
#         frame = np.transpose(frame,(2,0,1)) 
#         print("c")
#         frame = frame.reshape(1, frame.shape[0], frame.shape[1], frame.shape[2])
#         print("e")
#         frame = (frame / 255.0)
#         print("f")
#         #print(frame)
        
#         # Analyze the frame using the get_pose function
#         pose = dlc_live.get_pose(frame)
#         print("g")
        
#         # Store the pose for this frame
#         poses.append(pose)
#         print(pose["bodypart"]["poses"][0][0][0])
    

#         frame_index += 1
#         print(frame_index)
    
#     # Release the video capture object
#     cap.release()
    
#     return poses

In [1]:
def analyze_video2(video_path: str, dlc_live, pcutoff=0.5, display_radius=5, colors=None, resize=None, im_size=None):
    # Load video
    cap = cv2.VideoCapture(video_path)
    poses = []
    frame_index = 0

    # Get the video writer setup to save the output video with keypoints
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if resize:
        frame_width, frame_height = im_size
    vwriter = cv2.VideoWriter('output_with_keypoints.avi', fourcc, fps, (frame_width, frame_height))

    # Initialize default colors if not provided
    if colors is None:
        # Assuming there are 11 keypoints, adjust the length based on your model
        colors = [(255, 0, 0)] * 11  # Default to red for all keypoints; adjust if needed

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Prepare the frame for the model
        frame_for_model = np.array(frame, dtype=np.float32)
        frame_for_model = np.transpose(frame_for_model, (2, 0, 1))
        frame_for_model = frame_for_model.reshape(1, frame_for_model.shape[0], frame_for_model.shape[1], frame_for_model.shape[2])
        frame_for_model = (frame_for_model / 255.0)
        
        # Analyze the frame using the get_pose function
        pose = dlc_live.get_pose(frame_for_model)
        
        # Store the pose for this frame
        poses.append(pose)
        
        # Visualize keypoints on the frame
        this_pose = poses[-1]["bodypart"]["poses"][0]  # Adjust indexing as per your actual data structure
        for j in range(this_pose.shape[0]):
            confidence_tensor = this_pose[j, 2]  # Access the tensor with confidence values
            confidence = confidence_tensor[0].item()  # Extract the first confidence score as a scalar

            if confidence > pcutoff:
                x = int(this_pose[j, 0, 0].item())  # Assuming the first element corresponds to the x-coordinate
                y = int(this_pose[j, 1, 0].item())  # Assuming the first element corresponds to the y-coordinate

                # Draw circle only if color is valid and provided for this keypoint
                if j < len(colors):
                    frame = cv2.circle(frame, (x, y), display_radius, colors[j], thickness=-1)
                else:
                    frame = cv2.circle(frame, (x, y), display_radius, (0, 255, 0), thickness=-1)  # Default color green

        # Resize the frame if needed
        if resize is not None:
            frame = cv2.resize(frame, im_size)

        # Write the frame with keypoints to the output video
        vwriter.write(frame)

        frame_index += 1

    # Release the video capture and writer objects
    cap.release()
    vwriter.release()

    return poses


In [2]:
from dlclive import DLCLive, Processor
import dlclive
from dlclive.display import Display
import cv2
import numpy as np 

Loading DLC 3.0.0rc2...


/Users/annastuckert/anaconda3/envs/deeplabcut3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

dlc_proc = Processor()
dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt', model_type='pytorch') #change model type to 'pytorch' for raw .pt model

#very short video
video_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/1_20cms_0degUP_first_03s.avi'
#short video
#video_path = '/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/1_20cms_0degUP_first_1s.avi'

poses = analyze_video2(video_path, dlc_live)

In [ ]:
print(poses)